# Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.5 MB/s eta 0:00:00
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.5-py3-none-any.whl.metadata (4.7 kB)
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.7/160.7 kB 6.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain-0.1.20-py3-none-any.whl.metadata (13 kB)
  Using cached opentelemetry_exporter_otlp_proto_common-1.25.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached opentelemetry_proto-1.25.0-py3-none-any.whl.metadata (2.2 kB)
  Usin

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [3]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [7]:
import os
# from utils import get_openai_api_key, get_serper_api_key

openai_api_key = ""
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = 

## crewAI Tools

In [5]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

### Data Analyst

In [23]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze cryptocurrency market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions. ",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected cryptocurrency ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}. Specifically "
        "add relevant headings for key insights and use bullet points where possible."        
    ),
    agent=data_analyst_agent,
)

### Trading Strategy Developer

In [12]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


### Trade Advisor

In [24]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}. Use bullet points where possible."
    ),
    agent=execution_agent,
)


### Risk Advisor

In [26]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [27]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-4o", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

2024-07-16 14:57:57,852 - 8371518144 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

- Set the inputs for the execution of the crew.

In [29]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'BTCUSDT',
    'initial_capital': '100000',
    'risk_tolerance': 'High',
    'trading_strategy_preference': 'Long Term',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [30]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected cryptocurrency (BTCUSDT). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
To effectively monitor and analyze the BTCUSDT market, I need to leverage the expertise of my team members. The process will involve data analysis, developing trading strategies, getting trade advice, and assessing risks. I will start by delegating tasks to the appropriate team members and asking relevant questions to gather insights.

First, I need to gather the necessary data and perform a preliminary analysis. I will delegate this task to the Data Analyst.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Collect and analyze market data for BTCUSDT over the past six months to identify trends and patterns.", "context": "We need to continuously monitor BTCUSDT. Us

- Display the final result as Markdown.

In [31]:
from IPython.display import Markdown
Markdown(result)

### Comprehensive Risk Analysis Report for BTCUSDT Trading Strategies

#### 1. Day Trading and Technical Analysis
**Potential Risks:**
- **High Volatility:** BTCUSDT is known for its significant price fluctuations. This volatility can lead to quick and unexpected price swings, potentially resulting in losses if stop-loss orders are not triggered in time.
- **Technical Indicator Reliability:** While indicators like the 20-day EMA, 50-day SMA, RSI, Williams %R, and MACD can be useful, they are not foolproof. Market conditions can change rapidly, rendering these indicators less effective.
- **Execution Risk:** Entering near key support levels and exiting at resistance levels requires precise execution. Delays or errors in execution can lead to missed opportunities or losses.

**Mitigation Strategies:**
- **Set Tight Stop-Loss Orders:** Ensure stop-loss orders are set just below the support level to minimize potential losses.
- **Regular Monitoring:** Continuously monitor technical indicators and adjust strategies as needed.
- **Risk Capital:** Only use capital that you can afford to lose, given the high-risk nature of day trading.

#### 2. Automated Trading Strategies
**Potential Risks:**
- **Technical Errors:** Automated trading bots require continuous optimization to adapt to changing market conditions. Technical errors or malfunctions can lead to significant losses.
- **Over-Optimization:** Over-optimizing algorithms based on past data can lead to poor performance in future market conditions.
- **Market Anomalies:** Unpredictable market events can disrupt the algorithms, leading to unexpected losses.

**Mitigation Strategies:**
- **Regular Testing:** Continuously test and refine algorithms to ensure they align with current market trends.
- **Diversification of Bots:** Use multiple trading bots with different strategies to spread risk.
- **Fail-Safe Mechanisms:** Implement fail-safe mechanisms to shut down the bots in case of extreme market conditions.

#### 3. Capitalizing on Market Sentiment
**Potential Risks:**
- **Sentiment Reversal:** Positive market sentiment can quickly reverse due to unexpected news or events, leading to rapid price changes that may not align with the bullish sentiment.
- **Over-Reliance on News:** Relying too heavily on news events for trading decisions can be risky if the news is misinterpreted or its impact is overestimated.

**Mitigation Strategies:**
- **Diversified Information Sources:** Use multiple sources of information to gauge market sentiment accurately.
- **Quick Reaction Times:** Be prepared to act quickly in response to changing market sentiment.
- **Hedging:** Use hedging strategies to protect against sudden sentiment reversals.

#### 4. Diversification
**Potential Risks:**
- **Correlation Risk:** The correlation between altcoins and Bitcoin's price trends may not always provide effective diversification, especially during periods of high market volatility.
- **Liquidity Risk:** Some altcoins may have lower liquidity, making it difficult to execute trades without significant price impact.

**Mitigation Strategies:**
- **Diversify Across Uncorrelated Assets:** Invest in a mix of cryptocurrencies and other asset classes to reduce correlation risk.
- **Liquidity Assessment:** Regularly assess the liquidity of the assets in the portfolio and adjust holdings accordingly.
- **Regular Portfolio Rebalancing:** Periodically rebalance the portfolio to maintain desired risk levels.

#### 5. Risk Management Techniques
**Potential Risks:**
- **Market Conditions:** Rapid changes in market conditions can make it challenging to adjust risk management strategies in real-time, potentially leading to significant losses.
- **Implementation Errors:** Errors in implementing stop-loss orders, position sizing, or hedging strategies can negate their effectiveness.

**Mitigation Strategies:**
- **Dynamic Risk Management:** Continuously adjust risk management strategies based on current market conditions.
- **Education and Training:** Ensure all team members are well-trained in risk management techniques and their implementation.
- **Automated Risk Controls:** Use automated systems to implement and monitor risk management strategies in real-time.

### Conclusion
The proposed trading strategies for BTCUSDT involve significant risks due to the high volatility of the cryptocurrency market, the reliance on technical indicators, the potential for technical errors in automated systems, and the challenges of accurately gauging market sentiment. Effective risk management strategies, continuous monitoring, and regular adjustments are crucial to mitigate these risks and navigate the BTCUSDT market successfully.